In [2]:

import mne
import numpy as np
import matplotlib as plt
import pandas as pd
import os
import scipy.io

In [36]:
# define path
global project_path, eeg_path, data_path, label_path, post_fix, sampling
project_path = '/Volumes/CLPS_Watanabe_Lab/users/Zhiyan/SleepDecoding/'
eeg_path = 'Experiment/sbj/'
data_path = 'EEG/raw/'
label_path = 'label/'
post_fix = '_reref_01100.vhdr'
subjects = ['sldcd01', 'sldcd03', 'sldcd05','sldcd06']
naps = ['nap2', 'nap3']
sampling = 500 # hz



In [37]:
len_epochs = 30 # [s]
num_ch = 36

In [38]:
def load_label_raw(SUB, NAP):
    fn_path = project_path +  eeg_path + SUB + '/' + NAP + '/' +  label_path +  'stageMat_' + SUB + '_' + NAP 
    label = scipy.io.loadmat(fn_path)
    label = label['stageMat']
    return label

In [39]:
def load_bad_ch(SUB, NAP):
    fn_path = project_path +  eeg_path + SUB + '/' + NAP + '/' +  label_path + SUB + '_' + NAP + '_bad' + '.txt'
    bad = pd.read_csv(fn_path, sep=" ", header = None)
    return set(bad.iloc[:,0])

In [40]:
def load_parse_eeg(SUB, NAP):
    fn_path = project_path +  eeg_path + SUB + '/' + NAP + '/' +  data_path + SUB + NAP + post_fix
    raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=False)
    return raw

In [41]:
def create_annotations(label):
    len_recor = label[-1,0]
    start = np.arange(0,len_recor + 0.5, 0.5)
    start = start * 60
    duration = [30] * len(start) 
    description = label[:,1]
    id = label[:, 2] == 1
    annotations = mne.Annotations(start, duration, description)
    annotations.description[id] = 'bad'
    return annotations 

In [42]:
data_final = pd.DataFrame(columns=['alpha','theta','slowwave','sigma','channel_name', 'label', 'subject','NAP'])
for SUB in subjects:
    for NAP in naps:
        eeg_raw = load_parse_eeg(SUB, NAP)
        label = load_label_raw(SUB, NAP)
        bad = load_bad_ch(SUB, NAP)
        annotations = create_annotations(label)
        eeg_raw.set_annotations(annotations, emit_warning=False)
        annot_id = {'-1.0':-1, 
                    '0.0':0,
                    '1.0':1,
                    '2.0':2,
                    '3.0':3
                   }
        event_raw, _ = mne.events_from_annotations(eeg_raw, event_id = annot_id, chunk_duration=30.)
        tmax = 30. - 1. / eeg_raw.info['sfreq']
        epochs = mne.Epochs(eeg_raw, events = event_raw, tmin = 0, tmax= tmax, baseline = None)
        psds, freqs = mne.time_frequency.psd_welch(epochs, picks='eeg', fmin=0.5, fmax=30.)
        # create dataframe
        ch_names = eeg_raw.ch_names
        num_epochs = psds.shape[0]
        num_chs = psds.shape[1]
        label_s = label[label[:,2]==0, :]
        data_preprocessed_eeg = pd.DataFrame(0.0, index=range(num_epochs * num_chs), columns=['alpha','theta','slowwave','sigma','channel_name', 'label', 'subject','NAP'])
        data_preprocessed_eeg['subject'] = SUB
        data_preprocessed_eeg = data_preprocessed_eeg.astype({'channel_name':'str', 'label': 'int32', 'subject':'str'})
        
        alpha  = np.mean(psds[:,:,5:7], axis = 2)
        theta = np.mean(psds[:,:,2:5], axis = 2)
        slowwave = np.mean(psds[:,:,0:2], axis = 2)
        sigma = np.mean(psds[:,:,6:11],axis = 2)
        
        # alpha 
        alpha_sum = alpha.ravel('F')
        alpha_sum.reshape(-1,1)
        data_preprocessed_eeg.iloc[:, 0] = alpha_sum 
        # theta
        theta_sum = theta.ravel('F')
        theta_sum.reshape(-1,1)
        data_preprocessed_eeg.iloc[:, 1] = theta_sum 
        # slowwave 
        sw_sum = slowwave.ravel('F')
        sw_sum.reshape(-1,1)
        data_preprocessed_eeg.iloc[:, 2] = sw_sum
        # sigma
        sigma_sum = sigma.ravel('F')
        sigma_sum.reshape(-1,1)
        data_preprocessed_eeg.iloc[:, 3] = sigma_sum
        
        for i in range(num_chs):
            if ch_names[i] in bad:  
                data_preprocessed_eeg.iloc[i * num_epochs : i * num_epochs + num_epochs, 0:4] = np.nan
                continue
            data_preprocessed_eeg.iloc[i * num_epochs : i * num_epochs + num_epochs, 4] = ch_names[i]
            data_preprocessed_eeg.iloc[i * num_epochs : i * num_epochs + num_epochs, 5] = label_s[:,1]
        data_preprocessed_eeg['NAP'] = NAP
        data_final = data_final.append(data_preprocessed_eeg)

Extracting parameters from /Volumes/CLPS_Watanabe_Lab/users/Zhiyan/SleepDecoding/Experiment/sbj/sldcd01/nap2/EEG/raw/sldcd01nap2_reref_01100.vhdr...
Setting channel info structure...
DigMontage is a superset of info. 3 in DigMontage will be ignored. The ignored channels are: {'HEOG', 'EMG', 'VEOG'}
Used Annotations descriptions: ['-1.0', '0.0', '1.0', '2.0', '3.0']
173 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 173 events and 15000 original time points ...


<ipython-input-40-67f07061dceb>:3: RuntimeWarning: No coordinate information found for channels ['EMG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=False)
<ipython-input-40-67f07061dceb>:3: DeprecationWarning: The `montage` parameter from `read_raw_brainvision` is deprecated and will be removed  in version 0.20. Use  raw.set_montage(montage) instead.
  raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=False)


0 bad epochs dropped
Effective window size : 0.512 (s)
Extracting parameters from /Volumes/CLPS_Watanabe_Lab/users/Zhiyan/SleepDecoding/Experiment/sbj/sldcd01/nap3/EEG/raw/sldcd01nap3_reref_01100.vhdr...
Setting channel info structure...


<ipython-input-40-67f07061dceb>:3: RuntimeWarning: No coordinate information found for channels ['EMG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=False)


DigMontage is a superset of info. 3 in DigMontage will be ignored. The ignored channels are: {'HEOG', 'EMG', 'VEOG'}


...

0 bad epochs dropped
Effective window size : 0.512 (s)
Extracting parameters from /Volumes/CLPS_Watanabe_Lab/users/Zhiyan/SleepDecoding/Experiment/sbj/sldcd06/nap2/EEG/raw/sldcd06nap2_reref_01100.vhdr...
Setting channel info structure...
DigMontage is a superset of info. 3 in DigMontage will be ignored. The ignored channels are: {'HEOG', 'EMG', 'VEOG'}
Used Annotations descriptions: ['-1.0', '0.0', '1.0', '2.0', '3.0']
167 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 167 events and 15000 original time points ...


<ipython-input-40-67f07061dceb>:3: RuntimeWarning: No coordinate information found for channels ['EMG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=False)
<ipython-input-40-67f07061dceb>:3: DeprecationWarning: The `montage` parameter from `read_raw_brainvision` is deprecated and will be removed  in version 0.20. Use  raw.set_montage(montage) instead.
  raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=False)


0 bad epochs dropped
Effective window size : 0.512 (s)
Extracting parameters from /Volumes/CLPS_Watanabe_Lab/users/Zhiyan/SleepDecoding/Experiment/sbj/sldcd06/nap3/EEG/raw/sldcd06nap3_reref_01100.vhdr...
Setting channel info structure...
DigMontage is a superset of info. 3 in DigMontage will be ignored. The ignored channels are: {'HEOG', 'EMG', 'VEOG'}
Used Annotations descriptions: ['-1.0', '0.0', '1.0', '2.0', '3.0']
171 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 171 events and 15000 original time points ...


<ipython-input-40-67f07061dceb>:3: RuntimeWarning: No coordinate information found for channels ['EMG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=False)
<ipython-input-40-67f07061dceb>:3: DeprecationWarning: The `montage` parameter from `read_raw_brainvision` is deprecated and will be removed  in version 0.20. Use  raw.set_montage(montage) instead.
  raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=False)


0 bad epochs dropped
Effective window size : 0.512 (s)


In [44]:
data_final.to_csv('processed_sleep_fft.csv')